In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_upstage import ChatUpstage
from langgraph.graph import StateGraph, END
from typing import TypedDict

In [7]:
from sell_side_agents import SellSideAgent
from checklist_insepctor import ChecklistInspector
from langgraph.graph import StateGraph
from typing import TypedDict

# 상태 정의
class AgentState(TypedDict):
    history: list[str]
    turn: int
    checklist: dict
    sellside_context: str
    user_query: str

# 두 에이전트 초기화
sellside_agent = SellSideAgent(db_stub=lambda x: f"{x}의 회사 데이터입니다.", api_key=os.getenv("SOLAR_API_KEY"))
checklist_inspector = ChecklistInspector(api_key=os.getenv("SOLAR_API_KEY"))

# SellSideAgent 노드
def sellside_agent_node(state: AgentState) -> AgentState:
    sellside_context = sellside_agent.generate_context("동연컴퍼니", state["checklist"])
    return {
        **state,
        "history": state["history"] + [f"SellSideAgent: {sellside_context}"],
        "sellside_context": sellside_context,
        "turn": state["turn"] + 1,
    }

# ChecklistInspector 노드
def checklist_inspector_node(state: AgentState) -> AgentState:
    inspection_result = checklist_inspector.inspect_checklist(
        state["checklist"], state["sellside_context"], state["user_query"]
    )
    return {
        **state,
        "history": state["history"] + [f"ChecklistInspector: {inspection_result}"],
        "turn": state["turn"] + 1,
    }

# 대화 종료 조건
def check_done(state: AgentState) -> str:
    if "총점" in state["history"][-1] or state["turn"] >= 4:
        return "END"
    return "sellside" if state["turn"] % 2 == 0 else "inspector"

# StateGraph 정의
workflow = StateGraph(AgentState)
workflow.add_node("sellside", sellside_agent_node)
workflow.add_node("inspector", checklist_inspector_node)
workflow.set_entry_point("sellside")
workflow.add_conditional_edges("sellside", check_done)
workflow.add_conditional_edges("inspector", check_done)

graph = workflow.compile()

# 초기 상태
initial_state = {
    "history": ["안녕하세요! 동연컴퍼니의 투자유치 점검을 도와드리겠습니다. 회사 측 정보 중 필요한 것이 있다면 점검자님께서 요청해 주시고, 회사 측 전문가님께서는 성심성의껏 사실만을 대답해주시길 바랍니다."],
    "turn": 0,
    "checklist": {
        "업종": 20,
        "연매출": 15,
        "투자유치": 25,
        "부채비율": 30,
        "영업이익률": 10,
    },
    "sellside_context": "",
    "user_query": "헬스케어 산업에 종사하는, 연매출 100억 이상, 최근 1년 투자금 100억 이상, 작년 성장률 25% 이상, 영업이익률 10% 이상인 기업을 찾고 있습니다.",
}

# 실행
final_state = graph.invoke(initial_state)

# 결과 출력
print("\n\n🧠 전체 대화 로그:")
for msg in final_state["history"]:
    print(msg)



🧠 전체 대화 로그:
안녕하세요! 동연컴퍼니의 투자유치 점검을 도와드리겠습니다. 회사 측 정보 중 필요한 것이 있다면 점검자님께서 요청해 주시고, 회사 측 전문가님께서는 성심성의껏 사실만을 대답해주시길 바랍니다.
SellSideAgent: 
동연컴퍼니는 IT 및 소프트웨어 개발 업종에 속해 있습니다. 현재 동연컴퍼니의 연매출을 공개할 수는 없지만, 작년 대비 상당한 성장을 이루었으며, 안정적인 수익을 창출하고 있습니다. 동연컴퍼니는 현재까지 총 X억원 규모의 투자를 유치했으며, 이는 동종 업계 평균 이상의 수준입니다. 부채비율은 30% 미만으로 매우 낮은 수준을 유지하고 있으며, 이는 동연컴퍼니가 재무적으로 안정적인 상태를 유지하고 있음을 의미합니다. 영업이익률은 최근 몇 년간 꾸준한 증가세를 보이고 있으며, 현재 약 Y%의 높은 영업이익률을 기록하고 있습니다. 이러한 데이터는 동연컴퍼니의 재무적 안정성과 수익성을 반영하며, M&A 매도자로서 매력적인 조건을 제시합니다.
ChecklistInspector: ({'업종': 10, '연매출': 12, '투자유치': 20, '부채비율': 25, '영업이익률': 8}, 75)
SellSideAgent: 동연컴퍼니는 IT 서비스 및 소프트웨어 개발 업종에 종사하는 회사입니다. 연매출은 약 50억 원이며, 현재까지 총 20억 원의 투자를 유치했습니다. 부채비율은 약 50% 정도로 관리 가능한 수준이며, 영업이익률은 약 15%로 업계 평균과 비슷한 수준을 유지하고 있습니다. 이러한 컨텍스트를 제공함으로써 체크리스트 평가자는 동연컴퍼니의 M&A 매도자 위치를 더욱 정확하게 평가할 수 있을 것입니다.
ChecklistInspector: ({'업종': 12, '연매출': 10, '투자유치': 20, '부채비율': 25, '영업이익률': 10}, 77)
